##### About Dataset
Product Background:
A specific loan product X involves 3 parties.

Dairy Farmers

A Bank

A Food Company

The dairy farmers provide milk to the food company. The food company in return makes weekly payments to the farmers which are routed through the bank. The bank is offering loans to the dairy farmers against these payments. Duration of loan is 13 weeks. After a farmer has acquired the loan, it'll be repaid in weekly installments. These installments are deducted from payments of the food company to the farmer over the course of next 13 weeks.

Challenge:
Given historical 1 year (52 weeks) of payment data, the challenge is to identify:

Which farmers are most suitable to target for loan product X? In other words, which farmers are likely to have consistent payments in next 13 weeks?

How much loan should be offered to each farmer?

Business Context:
Farmers are not bound to provide milk to the food company. They can provide a portion of overall milk production to the food company, skip weeks or cease supply altogether whenever they want.

Farmers with high milk supply can be offered higher loans. Therefore it is preferable to target them whenever possible.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv(r"C:\Users\User\OneDrive\Desktop\PYTHON\farmers bank loan\farmers bank deposit to loan cleaned.csv")

In [3]:
df

,Farmer No,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,...,Week43,Week44,Week45,Week46,Week47,Week48,Week49,Week50,Week51,Week52
0,1,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,2918,1882,1939,1115
1,2,6487,10898,5133,3138,3056,5293,3170,2751,0.0,...,900,0,1931,122,0,2704,7147,5125,2589,917
2,3,1289,1392,1789,1319,662,2290,1994,2527,1172.0,...,2104,2306,1206,1417,1310,2153,1771,3527,3665,3110
3,4,3844,4256,3716,4511,5209,7299,9899,7440,1915.0,...,23350,26493,1217,2791,0,5203,7938,1909,850,546
4,5,3428,2346,2419,2110,1569,2110,1951,936,169.0,...,548,1216,657,225,357,0,0,181,189,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,1271,0,706,3507,908,1008,794,336,170,0.0,...,0,5160,5899,12465,13900,14840,10741,12040,9241,10365
1271,1272,174233,90237,65438,23628,2536,3241,2550,1662,977.0,...,6230,2736,2681,4051,2703,3651,4974,2088,2112,3358
1272,1273,9490,18144,1728,1573,0,0,1564,655,0.0,...,839,0,0,0,0,0,0,0,0,0
1273,1274,40052,16401,6779,24153,26360,5329,4225,696,0.0,...,0,0,0,0,0,13824,16870,15624,5414,9373


In [4]:
df.head()

,Farmer No,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,...,Week43,Week44,Week45,Week46,Week47,Week48,Week49,Week50,Week51,Week52
0,1,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,2918,1882,1939,1115
1,2,6487,10898,5133,3138,3056,5293,3170,2751,0.0,...,900,0,1931,122,0,2704,7147,5125,2589,917
2,3,1289,1392,1789,1319,662,2290,1994,2527,1172.0,...,2104,2306,1206,1417,1310,2153,1771,3527,3665,3110
3,4,3844,4256,3716,4511,5209,7299,9899,7440,1915.0,...,23350,26493,1217,2791,0,5203,7938,1909,850,546
4,5,3428,2346,2419,2110,1569,2110,1951,936,169.0,...,548,1216,657,225,357,0,0,181,189,76


In [5]:
df.tail()

,Farmer No,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,...,Week43,Week44,Week45,Week46,Week47,Week48,Week49,Week50,Week51,Week52
1270,1271,0,706,3507,908,1008,794,336,170,0.0,...,0,5160,5899,12465,13900,14840,10741,12040,9241,10365
1271,1272,174233,90237,65438,23628,2536,3241,2550,1662,977.0,...,6230,2736,2681,4051,2703,3651,4974,2088,2112,3358
1272,1273,9490,18144,1728,1573,0,0,1564,655,0.0,...,839,0,0,0,0,0,0,0,0,0
1273,1274,40052,16401,6779,24153,26360,5329,4225,696,0.0,...,0,0,0,0,0,13824,16870,15624,5414,9373
1274,1275,250,25455,196474,169923,113562,234901,161103,147797,24954.0,...,101991,9303,0,0,0,52732,42008,30330,28669,119138


#### Convert all week columns to numeric

In [6]:
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')


#### Handle missing values (example: fill NaNs with 0)

In [7]:
df.fillna(0, inplace=True)

#### Ensure 'Farmer No.' is an integer

In [8]:
df['Farmer No'] = df['Farmer No'].astype(int)

#### Standardize column names

In [9]:
df.columns = df.columns.str.strip().str.replace(' ', '_')

#### Remove duplicates

In [10]:
df.drop_duplicates(inplace=True)

#### Calculate the total payments for each farmer

In [11]:
df['Total_Payments'] = df.iloc[:, 1:].sum(axis=1)

In [12]:
print("Cleaned Data:")

Cleaned Data:


In [13]:
print(df.head())

   Farmer_No  Week1  Week2  Week3  Week4  Week5  Week6  Week7  Week8   Week9  \
0          1      0      0      0      0      0      0      0      0     0.0   
1          2   6487  10898   5133   3138   3056   5293   3170   2751     0.0   
2          3   1289   1392   1789   1319    662   2290   1994   2527  1172.0   
3          4   3844   4256   3716   4511   5209   7299   9899   7440  1915.0   
4          5   3428   2346   2419   2110   1569   2110   1951    936   169.0   

   ...  Week44  Week45  Week46  Week47  Week48  Week49  Week50  Week51  \
0  ...       0       0       0       0       0    2918    1882    1939   
1  ...       0    1931     122       0    2704    7147    5125    2589   
2  ...    2306    1206    1417    1310    2153    1771    3527    3665   
3  ...   26493    1217    2791       0    5203    7938    1909     850   
4  ...    1216     657     225     357       0       0     181     189   

   Week52  Total_Payments  
0    1115        373613.0  
1     917        5

In [14]:
df.to_csv('cleaned_data.csv', index=False)

In [15]:
print("Data saved to 'cleaned_data.csv'")

Data saved to 'cleaned_data.csv'


#### average weekly payment for each farmer: Columns: Farmer No., Week1 to Week52


In [16]:
# Calculate the average weekly payment for each farmer
df['Average_Weekly_Payment'] = df.iloc[:, 1:].mean(axis=1)

# Inspect the data with the new column
print("Data with Average Weekly Payment:")
print(df.head())

# Save the cleaned data with the average weekly payment to a new CSV file (optional)
df.to_csv('cleaned_data_with_average.csv', index=False)

print("Data saved to 'cleaned_data_with_average.csv'")

Data with Average Weekly Payment:
   Farmer_No  Week1  Week2  Week3  Week4  Week5  Week6  Week7  Week8   Week9  \
0          1      0      0      0      0      0      0      0      0     0.0   
1          2   6487  10898   5133   3138   3056   5293   3170   2751     0.0   
2          3   1289   1392   1789   1319    662   2290   1994   2527  1172.0   
3          4   3844   4256   3716   4511   5209   7299   9899   7440  1915.0   
4          5   3428   2346   2419   2110   1569   2110   1951    936   169.0   

   ...  Week45  Week46  Week47  Week48  Week49  Week50  Week51  Week52  \
0  ...       0       0       0       0    2918    1882    1939    1115   
1  ...    1931     122       0    2704    7147    5125    2589     917   
2  ...    1206    1417    1310    2153    1771    3527    3665    3110   
3  ...    1217    2791       0    5203    7938    1909     850     546   
4  ...     657     225     357       0       0     181     189      76   

   Total_Payments  Average_Weekly_Paymen

#### Standard deviation of the weekly payment

In [17]:
# Calculate the standard deviation of weekly payments for each farmer
df['Std_Dev_Weekly_Payment'] = df.iloc[:, 1:].std(axis=1)

# Inspect the data with the new column
print("Data with Standard Deviation of Weekly Payment:")
print(df.head())

# Save the cleaned data with the standard deviation to a new CSV file (optional)
df.to_csv('cleaned_data_with_std_dev.csv', index=False)

print("Data saved to 'cleaned_data_with_std_dev.csv'")

Data with Standard Deviation of Weekly Payment:
   Farmer_No  Week1  Week2  Week3  Week4  Week5  Week6  Week7  Week8   Week9  \
0          1      0      0      0      0      0      0      0      0     0.0   
1          2   6487  10898   5133   3138   3056   5293   3170   2751     0.0   
2          3   1289   1392   1789   1319    662   2290   1994   2527  1172.0   
3          4   3844   4256   3716   4511   5209   7299   9899   7440  1915.0   
4          5   3428   2346   2419   2110   1569   2110   1951    936   169.0   

   ...  Week46  Week47  Week48  Week49  Week50  Week51  Week52  \
0  ...       0       0       0    2918    1882    1939    1115   
1  ...     122       0    2704    7147    5125    2589     917   
2  ...    1417    1310    2153    1771    3527    3665    3110   
3  ...    2791       0    5203    7938    1909     850     546   
4  ...     225     357       0       0     181     189      76   

   Total_Payments  Average_Weekly_Payment  Std_Dev_Weekly_Payment  
0     

#### farmers have the most consistent weekly payment (least volatility)

In [18]:
# Sort the DataFrame by the standard deviation in ascending order
df_sorted = df.sort_values(by='Std_Dev_Weekly_Payment')

# Get the farmers with the least volatility
least_volatility_farmers = df_sorted[['Farmer_No', 'Std_Dev_Weekly_Payment']]

print("Farmers with the most consistent weekly payment (least volatility):")
print(least_volatility_farmers)

# Save the sorted data with standard deviation to a new CSV file (optional)
df_sorted.to_csv('sorted_farmers_by_std_dev.csv', index=False)
print("Data saved to 'sorted_farmers_by_std_dev.csv'")

Farmers with the most consistent weekly payment (least volatility):
      Farmer_No  Std_Dev_Weekly_Payment
140         141            0.000000e+00
1130       1131            0.000000e+00
707         708            7.812825e+01
402         403            4.460319e+02
929         930            4.834925e+02
...         ...                     ...
29           30            2.062365e+06
1247       1248            2.175971e+06
441         442            2.977247e+06
722         723            3.108106e+06
439         440            3.541001e+06

[1275 rows x 2 columns]
Data saved to 'sorted_farmers_by_std_dev.csv'


#### farmers have the highest average weekly production?

In [19]:
# Calculate the average weekly payment for each farmer
df['Avg_Weekly_Payment'] = df.iloc[:, 1:].mean(axis=1)

# Sort the DataFrame by the average weekly payment in descending order
df_sorted_avg = df.sort_values(by='Avg_Weekly_Payment', ascending=False)

# Get the farmers with the highest average weekly payment
highest_avg_payment_farmers = df_sorted_avg[['Farmer_No', 'Avg_Weekly_Payment']]

print("Farmers with the highest average weekly payment:")
print(highest_avg_payment_farmers)

# Save the sorted data with average weekly payment to a new CSV file (optional)
df_sorted_avg.to_csv('sorted_farmers_by_avg_payment.csv', index=False)
print("Data saved to 'sorted_farmers_by_avg_payment.csv'")

Farmers with the highest average weekly payment:
      Farmer_No  Avg_Weekly_Payment
439         440        1.041409e+06
722         723        9.185280e+05
441         442        8.793202e+05
1247       1248        6.393557e+05
29           30        6.053970e+05
...         ...                 ...
929         930        1.315366e+02
402         403        1.295958e+02
707         708        1.661091e+01
140         141        0.000000e+00
1130       1131        0.000000e+00

[1275 rows x 2 columns]
Data saved to 'sorted_farmers_by_avg_payment.csv'


#### Suitable Farmers for Loans by Identify farmers with consistent production and high average weekly production over the past 13 weeks. 
Columns: Farmer No., Week40 to Week52

#### Calculate the mean of the total standard deviation


In [20]:

mean_std_dev = df['Std_Dev_Weekly_Payment'].mean()

In [21]:
mean_avg_weekly_payment = df['Avg_Weekly_Payment'].mean()

In [22]:
print("Mean of the total standard deviation:", mean_std_dev)

Mean of the total standard deviation: 193168.863878164


In [23]:
print("Mean of the total average weekly payment:", mean_avg_weekly_payment)

Mean of the total average weekly payment: 56660.00920420191


#### Define thresholds for "consistent" and "high average". using average weekly payment and standard deviation of each farmer

In [30]:
# Convert all week columns to numeric, forcing any errors to be NaN
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# Handle missing values (example: fill NaNs with 0)
df.fillna(0, inplace=True)

# Calculate the standard deviation and average for Weeks 40 to 52
df['Std_Dev_Weekly_Payment'] = df.iloc[:, 1:].std(axis=1)
df['Avg_Weekly_Payment'] = df.iloc[:, 1:].mean(axis=1)

# Print the calculated averages and standard deviations
print("Calculated Averages and Standard Deviations:")
print(df[['Farmer_No', 'Avg_Weekly_Payment', 'Std_Dev_Weekly_Payment']])

# Define thresholds
mean_avg_weekly_payment >= 56705.151021223726
mean_std_dev >= 193168.863878164

# Identify suitable farmers for loan offers
suitable_farmers = df[(df['Avg_Weekly_Payment'] > mean_avg_weekly_payment) & (df['Std_Dev_Weekly_Payment'] < mean_std_dev)]

print("Suitable farmers for loan offers:")
print(suitable_farmers[['Farmer_No', 'Avg_Weekly_Payment', 'Std_Dev_Weekly_Payment']])

Calculated Averages and Standard Deviations:
      Farmer_No  Avg_Weekly_Payment  Std_Dev_Weekly_Payment
0             1        14764.957207            50747.657648
1             2        20003.254237            67470.243748
2             3        13630.974759            47505.591590
3             4       140563.466425           477652.006610
4             5         2673.260125             8990.974331
...         ...                 ...                     ...
1270       1271         5025.212174            17194.690001
1271       1272        75668.075258           259103.078619
1272       1273        18709.321335            64270.600976
1273       1274         9657.017136            33192.520869
1274       1275       157799.768585           530568.671450

[1275 rows x 3 columns]
Suitable farmers for loan offers:
     Farmer_No  Avg_Weekly_Payment  Std_Dev_Weekly_Payment
336        337        56994.843749           191001.745644
491        492        56842.251795           191471.298633

#### Suitatble farmers for loan using the mean average weekly bank deposit and mean  average standard deviatioh ( hish deposit with low standard deviation)

In [32]:
# Calculate the standard deviation and average for Weeks 40 to 52
df['Std_Dev_Weekly_Payment'] = df.iloc[:, 1:].std(axis=1)
df['Avg_Weekly_Payment'] = df.iloc[:, 1:].mean(axis=1)

# Define thresholds
mean_avg_weekly_payment = 56705.151021223726
mean_std_dev = 193168.863878164

# Identify suitable farmers for loan offers based on the new criteria
suitable_farmers = df[(df['Avg_Weekly_Payment'] <= mean_avg_weekly_payment) & (df['Std_Dev_Weekly_Payment'] <= mean_std_dev)].copy()

# Add a serial number column
suitable_farmers.reset_index(drop=True, inplace=True)
suitable_farmers.loc[:, 'Serial_No'] = suitable_farmers.index + 1

# Save the suitable farmers to a CSV file
suitable_farmers.to_csv('suitable_farmers.csv', index=False, columns=['Serial_No', 'Farmer_No', 'Avg_Weekly_Payment', 'Std_Dev_Weekly_Payment'])

print("Suitable farmers for loan offers with serial numbers:")
print(suitable_farmers[['Serial_No', 'Farmer_No', 'Avg_Weekly_Payment', 'Std_Dev_Weekly_Payment']])

Suitable farmers for loan offers with serial numbers:
     Serial_No  Farmer_No  Avg_Weekly_Payment  Std_Dev_Weekly_Payment
0            1          1        14764.948053            50747.538956
1            2          2        20003.242077            67470.087550
2            3          3        13630.966186            47505.479666
3            4          5         2673.258504             8990.953549
4            5          7        28627.548207            97981.943462
..         ...        ...                 ...                     ...
863        864       1269         3654.786659            12212.964336
864        865       1270        23410.366600            79942.691035
865        866       1271         5025.209073            17194.649882
866        867       1273        18709.309743            64270.450698
867        868       1274         9657.011149            33192.443235

[868 rows x 4 columns]


#### Suitable farmers for loan and amount in descending order

In [35]:
# Calculate the standard deviation and average for Weeks 40 to 52
df['Std_Dev_Weekly_Payment'] = df.iloc[:, 1:].std(axis=1)
df['Avg_Weekly_Payment'] = df.iloc[:, 1:].mean(axis=1)

# Define thresholds
mean_avg_weekly_payment = 56705.151021223726
mean_std_dev = 193168.863878164

# Identify suitable farmers for loan offers based on the new criteria
suitable_farmers = df[(df['Avg_Weekly_Payment'] <= mean_avg_weekly_payment) & (df['Std_Dev_Weekly_Payment'] <= mean_std_dev)].copy()

# Add a serial number column
suitable_farmers.reset_index(drop=True, inplace=True)
suitable_farmers.loc[:, 'Serial_No'] = suitable_farmers.index + 1

# Define a function to calculate the suitable loan amount based on average weekly payment
def calculate_loan_amount(avg_weekly_payment):
    # Example: loan amount is 10 times the average weekly payment
    return avg_weekly_payment * 10

# Calculate the suitable loan amount for each farmer
suitable_farmers['Suitable_Loan_Amount'] = suitable_farmers['Avg_Weekly_Payment'].apply(calculate_loan_amount)

# Sort the suitable farmers by the suitable loan amount in descending order
suitable_farmers = suitable_farmers.sort_values(by='Suitable_Loan_Amount', ascending=False)

# Save the suitable farmers to a CSV file
suitable_farmers.to_csv('suitable_farmers_with_loan_amounts_desc.csv', index=False, columns=['Serial_No', 'Farmer_No', 'Avg_Weekly_Payment', 'Std_Dev_Weekly_Payment', 'Suitable_Loan_Amount'])

print("Suitable farmers for loan offers with serial numbers and loan amounts (in descending order):")
print(suitable_farmers[['Serial_No', 'Farmer_No', 'Avg_Weekly_Payment', 'Std_Dev_Weekly_Payment', 'Suitable_Loan_Amount']])

Suitable farmers for loan offers with serial numbers and loan amounts (in descending order):
     Serial_No  Farmer_No  Avg_Weekly_Payment  Std_Dev_Weekly_Payment  \
134        135        186        56288.734733           188436.863121   
851        852       1245        56180.384992           190302.533045   
44          45         65        56129.739916           189430.487818   
521        522        800        56102.543616           188081.408334   
580        581        871        55380.518466           185424.213738   
..         ...        ...                 ...                     ...   
628        629        930          131.417502              476.943722   
257        258        403          129.483921              439.879820   
458        459        708           16.592864               77.135835   
100        101        141            0.000000                0.000000   
763        764       1131            0.000000                0.000000   

     Suitable_Loan_Amount  
13